In [1]:
import os
import pandas as pd
import glob
from tqdm import tqdm
from scipy.io import wavfile
from scipy import signal
import numpy as np
import soundfile as sf
os.chdir('../src/')
import config
import matplotlib.pyplot as plt
from tqdm import tqdm
import IPython.display as ipd
import json
import img.augmentation as aug
from sklearn.metrics import accuracy_score
import json
def read_test_predictions(folder):
    predictions = pd.read_csv(config.OUTPUT_FOLDER + '%s/pred_test.csv'%folder)
    predictions['target'] = predictions.drop('id', 1).idxmax(axis = 1)
    predictions["prob_%s"%folder] = predictions[[str(x) for x in np.arange(32)]].max(axis = 1)
    predictions['target'] = predictions['target'].map({str(v):k for k,v in config.mapping_dict.iteritems()})
    predictions.rename(columns = {'target' : 'target_%s'%folder}, inplace = True)
    gt_test = pd.read_csv(config.DATA_FOLDER + "gt_test.csv")
    predictions = predictions.merge(gt_test, on = 'id')
    assert len(predictions) == len(gt_test)
    
    return predictions[['id', 'target_%s'%folder, "prob_%s"%folder]]

In [2]:
nn1 = read_test_predictions("nn_21")
nn2 = read_test_predictions("nn_23")
nn3 = read_test_predictions("nn_24")

df = nn1.merge(nn2, on = "id")
df = df.merge(nn3, on = "id")
assert len(df) == len(nn1)

In [3]:
zz = (df[(df['target_nn_21'] == 'unknown') & (df['target_nn_23'] == 'unknown') & (df['target_nn_24'] == 'unknown')])

In [4]:
zz['prob'] = (zz['prob_nn_21'] + zz['prob_nn_23'] + zz['prob_nn_24']) / 3

In [5]:
submit53 = pd.read_csv("/home/tony/Downloads/submit70.csv")
submit53.rename(columns = {'label' : 'label_sub53'}, inplace = True)

In [21]:
ids = ([x + '.wav' for x in zz['id'].tolist()])
submit53.loc[submit53["fname"].isin(ids)]['label_sub53'].value_counts()

unknown    11444
on           845
left         568
no           495
right        475
down         313
go           236
up           210
off          129
stop          74
yes           19
Name: label_sub53, dtype: int64

In [28]:

submit53[submit53['fname'].isin(ids)]['label'].value_counts()

unknown    14808
Name: label, dtype: int64

In [22]:
submit53.loc[submit53["fname"].isin(ids), 'label_sub53'] = "unknown"
submit53.rename(columns = {'label_sub53':'label'},inplace = True)

In [31]:
submit53.to_csv("/home/tony/Downloads/submit70_unknowns_from_max_vote_21_23_24.csv", index = False)

In [29]:
submit53['label'].value_counts()

unknown    81061
silence    14116
on          6640
no          6589
right       6543
up          6521
go          6481
off         6334
left        6308
stop        6065
down        5986
yes         5894
Name: label, dtype: int64

In [11]:
label = 'stop'
ids_list = submit53[(submit53['fname'].isin([x + '.wav' for x in zz['id'].tolist()])) & (submit53['label_sub53'] == label)]['fname'].tolist()

In [14]:
N = 60

name = ids_list[N]
path = "/home/tony/ssd_buffer/kaggle_speech/test/audio/%s"%name
ipd.Audio(path)

In [ ]:



with open("unknown_unknowns_ids.json", 'w') as f:
    json.dump({'ids' : ids},f)